In [8]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the CRUD module
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Connect to database
username = "aacuser"
password = "MegaPassword987!"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# Retrieve initial data
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Load Grazioso Salvare Logo
try:
    image_filename = 'Grazioso Salvare Logo.png'
    encoded_image = base64.b64encode(open(image_filename, 'rb').read())
except FileNotFoundError:
    # Fallback if logo is not found
    encoded_image = base64.b64encode(b'')
    print(f"Logo file {image_filename} not found")

# Dashboard Layout
app.layout = html.Div([
    # Logo and Title
    html.Div([
        html.A(
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={'height': '100px', 'margin-right': '20px'}
            ) if encoded_image else html.Div("Logo Not Found"),
            target='_blank'
        ),
        html.H1('Grazioso Salvare Animal Shelter Dashboard', style={'display': 'inline-block', 'vertical-align': 'top'}),
        html.H3('Created by Fernando Balen', style={'color': 'gray'})
    ], style={'textAlign': 'center', 'marginBottom': '20px'}),

    # Interactive Filtering Options
    html.Div([
        html.Label('Select Rescue Type:', style={'fontWeight': 'bold'}),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster/Individual Tracking', 'value': 'tracking'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'inline-block', 'marginRight': '20px'}
        )
    ], style={'textAlign': 'center', 'marginBottom': '20px'}),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        style_table={'height': '400px', 'overflowY': 'auto'},
        style_cell={'textAlign': 'left'},
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        row_selectable='single',
        page_size=10
    ),

    # Charts Container
    html.Div(className='row', style={'display': 'flex', 'marginTop': '20px'}, children=[
        # Breed Distribution Chart (Left Column)
        html.Div(
            id='graph-id',
            className='col s12 m6',
            style={'width': '50%'}
        ),
        # Geolocation Map (Right Column)
        html.Div(
            id='map-id',
            className='col s12 m6',
            style={'width': '50%'}
        )
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Filter Data Table Based on Rescue Type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # Define queries based on rescue types and chosen breeds
    if filter_type == 'water':
        query = {
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'mountain':
        query = {
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'tracking':
        query = {
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    else:  # reset
        query = {}
    
    filtered_df = pd.DataFrame.from_records(db.read(query))
    filtered_df.drop(columns=['_id'], inplace=True)
    return filtered_df.to_dict('records')

# Update Breed Distribution Chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_breed_chart(viewData):
    if viewData is None or len(viewData) == 0:
        return [html.Div("No data available for chart", style={'textAlign': 'center', 'color': 'red'})]
    
    try:
        dff = pd.DataFrame.from_dict(viewData)
        
        # Get the top 10 breeds by count
        breed_counts = dff['breed'].value_counts()
        top_breeds = breed_counts.head(10).index.tolist()
        
        # Filter the DataFrame to only include the top 10 breeds
        dff_top = dff[dff['breed'].isin(top_breeds)]
        
        return [
            dcc.Graph(
                figure=px.pie(
                    dff_top, 
                    names='breed', 
                    title='Breed Distribution',
                    hole=0.3  # Creates a donut chart
                )
            )
        ]
    except Exception as e:
        return [html.Div(f"Error creating chart: {str(e)}", style={'textAlign': 'center', 'color': 'red'})]

# Update Geolocation Map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    # Check if viewData is None or empty
    if viewData is None or len(viewData) == 0:
        return [
            html.Div("No data available for mapping", 
                     style={'textAlign': 'center', 'color': 'red', 'padding': '20px'})
        ]
    
    # Convert to DataFrame
    try:
        dff = pd.DataFrame.from_dict(viewData)
    except Exception as e:
        return [
            html.Div(f"Error converting data: {str(e)}", 
                     style={'textAlign': 'center', 'color': 'red', 'padding': '20px'})
        ]
    
    # Find latitude and longitude columns
    possible_lat_cols = [col for col in dff.columns if 'lat' in col.lower()]
    possible_lon_cols = [col for col in dff.columns if 'lon' in col.lower()]
    
    if not possible_lat_cols or not possible_lon_cols:
        return [
            html.Div("No latitude/longitude columns found", 
                     style={'textAlign': 'center', 'color': 'red', 'padding': '20px'})
        ]
    
    # Use the first found latitude and longitude columns
    lat_col = possible_lat_cols[0]
    lon_col = possible_lon_cols[0]
    
    # Default to first row if no selection
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]
    
    # Validate coordinates
    try:
        lat = float(dff.iloc[row][lat_col])
        lon = float(dff.iloc[row][lon_col])
    except (ValueError, TypeError, IndexError) as e:
        return [
            html.Div(f"Invalid coordinates: {str(e)}", 
                     style={'textAlign': 'center', 'color': 'red', 'padding': '20px'})
        ]
    
    return [
        dl.Map(
            style={'width': '100%', 'height': '500px'}, 
            center=[lat, lon],  # Use selected animal's coordinates
            zoom=10, 
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Tooltip(str(dff.iloc[row, 4]) if 4 < len(dff.columns) else "Unknown Breed"),
                        dl.Popup([
                            html.H3("Animal Details"),
                            html.P(f"Name: {str(dff.iloc[row, 9]) if 9 < len(dff.columns) else 'Unknown'}"),
                            html.P(f"Breed: {str(dff.iloc[row, 4]) if 4 < len(dff.columns) else 'Unknown'}")
                        ])
                    ]
                )
            ]
        )
    ]

# Run the server
app.run_server(debug=True)

Dash app running on http://127.0.0.1:13183/
